In [72]:
import math

from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider, Text, HBox, FloatLogSlider
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation



In [73]:
history = pd.DataFrame(columns=['setpoint', 'p', 'i', 'd', 'f', 'steady state error', 'settling time', 'overshoot'])

In [74]:
#@title
def pid(setpoint, initial, p, i, izone, d, ff, times, model, v=12):
  err_acc = 0
  last_err = 0
  last_time = None
  position = initial
  velocity = 0
  for time in times:
    err = setpoint - position
    Vf = v * ff(setpoint, position)
    Vp = v * p * err
    if last_time is not None:
      d_time = (time - last_time)
      d_err = (err - last_err) / d_time
      Vd = v * d * d_err
      if abs(err) < izone:
        err_acc += err * d_time
      else:
        err_acc = 0
      Vi = v * i * err_acc
    else:
      (Vd, Vi, d_err, d_time) = (0, 0, 0, times[1] - times[0])
    V = Vf + Vp + Vd + Vi
    V = v if V > v else -v if V < -v else V

    position, velocity, a_f, a_m, a_g = model(position, velocity, d_time, V)
    yield (setpoint, position, velocity, V, Vf, Vp, Vi, Vd, a_f, a_m, a_g, err, d_err, err_acc)
    (last_time, last_err) = (time, err)

In [75]:
def ff_arm(f):
  return lambda setpoint, position: (f * math.cos(setpoint * math.pi / 180))

In [76]:
def model_arm(position, velocity, ts, V):
  inertia = 0.5
  mass = 0.1
  g = 10
  torque_per_volt = 1.0 
  acceleration_from_gravity = -g * mass * math.cos(math.pi * position / 180)
  acceleration_from_motor = deadzone(V) * torque_per_volt / inertia
  acceleration_from_friction = - 0.1 * velocity - 0.1 * (1 if velocity > 0 else -1)
  acceleration = acceleration_from_friction + acceleration_from_motor + acceleration_from_gravity
  new_velocity = (velocity + acceleration * ts)
  new_position  = position + ts * (velocity + new_velocity)/2 
  new_position = new_position-360 if new_position > 180 else new_position+360 if new_position < -180 else new_position
  return (new_position, new_velocity, acceleration_from_friction, acceleration_from_motor, acceleration_from_gravity)

def deadzone(V, deadzone=1):
  if V > deadzone:
    return (V-deadzone)*12/(12 - deadzone)
  elif V < -deadzone:
    return (V+deadzone)*12/(12 - deadzone)
  else:
    return 0

In [77]:
def analyze(df):
  max_error = df.err.abs().max()
  end = int(len(df)*0.98) # last 10% of data
  #max_end_error = df.err.iloc[end:].abs().max()
  initial_error = df.iloc[0].err
  steady_state_error = abs(df.err.iloc[end:].mean())
  if steady_state_error < initial_error * 0.5:
    try:
      settling_time = df[(df.err - steady_state_error).abs().gt(0.02 * abs(steady_state_error - initial_error))].index[-1]
    except IndexError:
      settling_time = df.index[-1]
    least_error = df.err.min()
    greatest_error = df.err.max()
    overshoot = abs((least_error - steady_state_error) / (initial_error - steady_state_error))
    return(steady_state_error, settling_time, overshoot)
  else:
    return (None, None, None)

In [78]:
def get_df(setpoint, p, i, d, f, t, ts):
  initial = -90
  times = np.arange(0, t, ts)
  izone = 20
  ff = ff_arm(f)
  return pd.DataFrame(columns=["setpoint", "position", "velocity", "V", "Vf", "Vp", "Vi", "Vd", "acc_f", "acc_m", "acc_g", "err", "delta_err", "total_err"],
                    data = pid(setpoint, initial, p, i, izone, d, ff, times, model_arm),
                    index=times)

def make_suggestion(setpoint, p, i, d, f, t, ts):
  def score(setpoint, p, i, d, f):
    df = get_df(setpoint, p, i, d, f, t, ts)
    return analyze(df)

  def gen():
    for p2 in np.arange(0, 0.1, 0.001):
      if abs(p2-p)>0.000001:
        yield (f'p={p2:.4f}', score(setpoint, p2, i, d, f))
    for i2 in np.arange(0, 0.01, 0.0001):
      if abs(i2-i)>0.000001:
        yield (f'i={i2:.4f}', score(setpoint, p, i2, d, f))
    for d2 in np.arange(0, 1, 0.01):
      if abs(d2-d)>0.000001:
        yield (f'd={d2:.4f}', score(setpoint, p, i, d2, f))
    for f2 in np.arange(0, 1, 0.01):
      if abs(f2-f)>0.000001:
        yield (f'f={f2:.4f}', score(setpoint, p, i, d, f2))
    yield('no change', score(setpoint, p, i, d, f))

  n_steps = t/ts
  if n_steps > 200:
    return "ERROR: increase ts or reduce t"
  suggestions = [ x for x in gen() if x[1][0] is not None ]
  #print(suggestions)
  best_suggestions = [ x[0] for x in (min(suggestions, key=lambda x: x[1][i]) for i in range(3)) ]
  return ", ".join(best_suggestions)

In [79]:
line_data = [ ('r', 'setpoint'), ('b', 'initial'), ('g', 'position') ]

def animate(setpoint, df):
  def aux(line, position):
    theta = position*math.pi/180
    x = math.cos(theta)
    y = math.sin(theta)
    line.set_data((0, x), (0, y))

  def aux2(frame):
      line = lines2
      pos = df.pos[frame]
      aux(line, pos)
      return line

  fig, ax = plt.subplots()
  lines = [
           (ax.plot([], color=color, label=label))[0]
           for color, label in line_data ]
  ax.set_xlim(-1, 1)
  ax.set_ylim(-1, 1)
  aux(lines[0], setpoint)
  aux(lines[1], df.pos[0])
  anim = FuncAnimation(fig, 
                       lambda frame: aux(lines[2], df.pos[frame]),
                       frames=len(df), interval=100)
  plt.show()

def plot_point(degrees, color, legend):
  theta = degrees * math.pi / 180
  lines = plt.plot(x=[0, math.cos(theta)], y=[0, math.sin(theta)], color=color, label=legend)
  return lines[0]


In [80]:

@interact(
    setpoint=IntSlider(min=-90, max=+90, layout=Layout(width='auto')), 
    p=FloatSlider(min=0., max=0.1, step=0.001, readout_format='0.4f', layout=Layout(width='auto')), 
    i=FloatSlider(min=0., max=0.01, step=0.0001, readout_format='0.4f', layout=Layout(width='auto')), 
    d=FloatSlider(min=0., max=1., step=0.01, readout_format='0.4f', layout=Layout(width='auto')), 
    f=FloatSlider(min=0., max=1., step=0.01, readout_format='0.4f', layout=Layout(width='auto')),
    t=IntSlider(value=30, min=10, max=120, step=1, readout_format='d', layout=Layout(width='auto')),
    ts=FloatLogSlider(value=1., min=-2, max=0., step=0.1, readout_format='0.2f', layout=Layout(width='auto')),
    )
def f(p=0, i=0, d=0, f=0, suggestions=False, extra_graphs=False, setpoint=0, t=30, ts=1):
  #print(dict(setpoint=setpoint, p=p, i=i, d=d, f=f))
  df = get_df(setpoint, p, i, d, f, t, ts)
  #animate(setpoint, df)
  nrows=7 if extra_graphs else 2
  fig, axes = plt.subplots(nrows=nrows, figsize=(30,5*nrows))  
  df[["setpoint", "position"]].plot(ax=axes[0], grid=True)
  df[["V", "Vf", "Vp", "Vi", "Vd"]].plot(ax=axes[1], ylim=(-12, 12), grid=True)
  if extra_graphs:
    df[["velocity"]].plot(ax=axes[2], grid=True)
    df[["acc_f", "acc_m", "acc_g"]].plot(ax=axes[3], grid=True)
    df[["err"]].plot(ax=axes[4], ylim=(-180,180), grid=True)
    df[["delta_err"]].plot(ax=axes[5], grid=True)
    df[["total_err"]].plot(ax=axes[6], grid=True)
  plt.show()

  (steady_state_error, settling_time, overshoot) = analyze(df)
  history.loc[len(history)] = [setpoint, p, i, d, f, steady_state_error, settling_time, overshoot]
  #display(history.tail())
  if suggestions: 
    suggestion = make_suggestion(setpoint, p, i, d, f, t, ts)
  else:
    suggestion = "disabled"
  steady_state_error_str = f"{steady_state_error:.6f}°" if steady_state_error is not None else "Undefined"
  settling_time_str = f"{settling_time:.6f}s" if settling_time is not None else "Undefined"
  overshoot_str = f"{overshoot:0.6%}" if overshoot is not None else "Undefined"
  display(HBox([Text(value=steady_state_error_str, description="Steady State Error", disabled=True),
    Text(value=settling_time_str, description="Settling Time", disabled=True),
    Text(value=overshoot_str, description="Overshoot", disabled=True), 
    Text(value=suggestion, description="Suggestion", disabled=True)],
    layout=Layout(width='auto')))
  









  

interactive(children=(FloatSlider(value=0.0, description='p', layout=Layout(width='auto'), max=0.1, readout_fo…